In [22]:
# !pip install datasets
# !pip install langchain
# !pip install langchain_community
# !pip install langchain_huggingface
# !pip install faiss-cpu
# !pip install langchain_openai
# !pip install -qU langchain_ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
from datasets import load_dataset

/home/dnl/projects/rag-project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("kuznetsoffandrey/sberquad", cache_dir="../data")

In [4]:
from langchain.schema import Document

In [5]:
documents = []

for example in dataset['train']:
    document = Document(
        page_content=example['context'],  
        metadata={
            'question': example['question'],  
            'answer': example['answers']['text'][0],  
            'id': example['id']
        }
    )
    documents.append(document)

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/dnl/projects/rag-project/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
from langchain_community.vectorstores import FAISS

In [8]:
vector_store = FAISS.from_documents(documents, embeddings)

vector_store.save_local("faiss_sberquad_index")
